## Custom Validation Process
### Running inference on video (or multiple videos) of a buoy being reeled towards the camera. The video has been exported to frames in the naming convention of 'frame_xxxxx.png'. A csv file contains the frame numbers and the respectiver distance to the buoy, as announced in the video. These numbers are then interpolated to assign a distance for each frame.
### The images are then passed to YOLO where they are downsampled to the specified sizes and inference is run on them. The results are saved, and compare to the hand annotated images, and if the boundary boxes match, pthey are deemed 'correct'. The confidence of the correct values are plotted as a function of distance. which is interpolated from the video. 

### Here's the file structure the the annotated videos should be in for the script to work as intended
```
annotated_videos/
    -video1/
        -frames_dist_coor.csv # csv of frames and the cooresponding distances
        -images/
            -frame_00001.png
            -frame_00002.png
            -..... 
        -labels/ # groundtruth/hand annotated labels
            -frame_00001.txt
            -frame_00002.txt
            -frame_...
        -inference/ # where results will be saved
    -video2/
```
        
Running the cells below will populate the inference folder like so:
```
        -inference/
            -imgsz1/
                -frame_00001.png #inference results w bb_box
                -frame_XXXX
                -frame_...
                -inference_results.json
                -validated_results.json
            -imgsz2/
                -frame...
                -inference_results.json
                -validated_results.json
```        

In [2]:
# Imports
import val
import custom_funcs as cf
import matplotlib.pyplot as plt
import os, torch, cv2, shutil, json, csv
import numpy as np
import pandas as pd
from utils.metrics import bbox_iou
import utils.general as gen
from matplotlib.lines import Line2D



Declare all your paths and img sizes

In [26]:
# path to yolo folder and weights of choice
dhufish_path ='/home/field/jennaDir/ccom_yolov5'
vm_path = '/home/jennaehnot/Desktop/ccom_yolov5'

weights_path= 'ccom_yolov5/model_testing/model3_4_best.pt'
yolo_dir = dhufish_path

# path to annotated videos like shown above
annotated_dir = '/home/jennaehnot/Desktop/annotated_videos'
buoy_vids = ['mooringBall_115cmH', 'lobsterPot_140cmH','redBall_150cmH']
plot_titles = ['Mooring Ball', 'Lobster Pot', 'Red Mooring Ball'] # for plot titles while saving
plot_save_name = '_conf_by_dist_by_imgsz.png' # will be added on to buoy_vid name

#img sizes to run inference at
img_sz = [640, 960,1280, 1600, 1920]
images = 'images'
labels = 'labels'
inference = 'inference'

#### Step 1: Running inference on all videos at all img sizes and saving results to json files

In [ ]:
# load model !

model = torch.hub.load(yolo_dir, 'custom', path = weights_path, source='local',force_reload=True)

for video in buoy_vids:

    # path to images
    img_dir = os.path.join(annotated_dir, video + images)
    img_filenames=os.listdir(img_dir)
    img_filenames.sort() # put them in chronological order

    # path to save inference dir
    inf_dir = os.path.join(annotated_dir, video, inference)

    # gt path
    gt_lbl_dir = os.path.join(annotated_dir, video, labels)

 

    for sz in img_sz: #f or every img size we want to run inference at
        # make a new folder within the inference folder for each image size
        imgsz_dir = 'imgsz' + str(sz)
        img_save_dir = os.path.join(inf_dir, imgsz_dir)
        save_json_path = os.path.join(img_save_dir, 'inference_results.json')
        output_stats = {}

        for file in img_filenames:
            # run inference
            img_path = os.path.join(img_dir, file)
            results = model(img_path, size= sz)
            results.save(labels=True, save_dir=img_save_dir,exist_ok =True) #exist_ok will rewrite existing folders instead of creating a new one so beware !
            
            times = list(results.t)
            times.append(sum(times)) # times = [prep_t, infer_t, nms_t, total]
            detects = results.pandas().xywhn[0]
            detects = detects.values.tolist()
            _, _, img_w, img_h = results.s

            frame_num = cf.imgname2frame(file)
            output_stats[file] = {
                'Frame': frame_num,
                'Img Dimensions': [sz, img_w, img_h],
                'Time Stats': times,
                'Detections': detects
                
            }
        
        # save stats 
        with open(save_json_path,'w') as json_file:
            json.dump(output_stats,json_file)
    

#### Step 2: Comparing inference results to ground truth (manually annotated) labels 
Labels validated by boundary box IOU, and does not take into account correct class prediction. Correct labels will be saved to another json file.

In [ ]:
# define minimum IOU threshold
miniou = 0.2

for video in buoy_vids:
    inf_dir = os.path.join(annotated_dir, video, inference)
    
    for sz in img_sz:

        pred_json = os.path.join(inf_dir,  'imgsz' + str(sz), 'inference_results.json')  
        
        files = os.listdir(gt_lbl_dir)
        files.sort()
        correct = [] 
        incorrect = []
        validated={}

        with open(pred_json, 'r') as jsonfile:
            pred_data = json.load(jsonfile)
    
        for img_name in img_filenames: 
            frame_num = cf.imgname2frame(img_name)
            gt_lbl_path = os.path.join(gt_lbl_dir, img_name[:-4] + '.txt')
            gt_labels = cf.load_labels(gt_lbl_path)
            if gt_labels: #if theres an object in the img
                ## inf = cf.Inference(pred_data[img_name]) #inference data for that img
                inf = pred_data[img_name]
                if inf['Detections xywhn:']: # if the model detected something
                    detects = inf['Detections xywhn:']
                    # do iou for between all labels
                    for i in range(0,len(gt_labels)):
                        gt_xywhn = gt_labels[i][1:]
                        for detect in detects:
                            d_xywhn = detect[0:4]

                            # do iou
                            if cf.compare_labels(gt_xywhn,d_xywhn, miniou): # compare labels returns false if boxes don't line up
                                validated[img_name] = inf
                                correct.append(img_name)
                            else:
                                incorrect.append(img_name)

        save_json_path = inf_dir +  'imgsz' + str(sz) + '/validated_results.json'
        with open(save_json_path,'w') as json_file:
            json.dump(validated,json_file)





#### Step 3: Plot the house down boots !


In [36]:
for q in range(0,len(buoy_vids)):
    video = buoy_vids[q]
    inf_dir = os.path.join(annotated_dir, video, inference)
    csvpath = os.path.join(annotated_dir, video, 'frames_dist_corr.csv')
    plot_save_path = os.path.join(inf_dir, video + plot_save_name)
    frames, dists = cf.frames2distances(csvpath) # get the interpolated dist for each frame num
    figure, axis = plt.subplots(len(img_sz), 1, figsize=(12,10), sharex=True, sharey=True)

    for i in range(0,len(img_sz)):

        validated_path = os.path.join(inf_dir, 'imgsz' + str(img_sz[i]), 'validated_results.json' )   
        # plot validated results for all the frames in the frames array

        with open(validated_path, 'r') as jsonfile:
                val_data = json.load(jsonfile) 
        conf = np.empty((0,len(frames)))
        times = np.empty((0,len(frames)))
        clss = np.empty((0,len(frames)))

        for j in frames:
            #make file name
            img_name = 'frame_' + str(f"{j:05d}") + '.png'
            try:
                det = val_data[img_name]['Detections']
                c = det[0][4]
                conf = np.append(conf, c)

                id = det[0][5]
                clss = np.append(clss, id)

                t = val_data[img_name]['Time Stats']
                total_t = t[3]
                times = np.append(times,total_t)

            except KeyError:
                #print(f"{img_name} did not have a detection in it")
                conf = np.append(conf, np.nan)
                times = np.append(times, np.nan)
                clss = np.append(clss, -1)

        color_map = {0: 'limegreen', 1: 'magenta', 2: 'royalblue', -1:'white'}
        colors = np.array([color_map[val] for val in clss])

        avg_t = np.nanmean(times)
        axis[i].scatter(dists,conf,s=2, c=colors)
        axis[i].grid(True, which='both', linestyle='--', color='gray', linewidth=0.5,alpha=0.5)
        axis[i].invert_xaxis()
        axis[i].set_ylim(0,1.0)
        axis[i].set_xlim(100,4)
        axis[i].set_yticks([0, 0.25, 0.5, 0.75, 1])
        axis[i].set_xticks(np.linspace(100,5,20))
        axis[i].set_title(f"Input Size = {img_sz[i]}," + r'  $\bar{t} = $'+ f"{avg_t:.2f} ms", fontsize = 10) 
        #axis[i].text(0.01, 0.95, f'Avg. t = {avg_t:.2f} ms', transform= axis[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='left')

    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='limegreen', markersize=10, label='0: navBuoy'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='magenta', markersize=10, label='1: mooringBall'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='royalblue', markersize=10, label='2: fishingBuoy')
    ]

    # Add the legend to the plot
    title = plot_titles[q]
    figure.suptitle(f'Detection of {title} at Different Compression Sizes', fontsize=18, y=0.96)
    figure.text(0.5, 0.03, 'Buoy Distance from Camera', ha='center', va='center', fontsize=18)
    figure.text(0.03, 0.5, 'Confidence', ha='center', va='center', rotation='vertical', fontsize=18)
    figure.legend(handles=legend_elements, loc='center', ncol=3, bbox_to_anchor=(0.5, 0.91))
    plt.tight_layout(rect=[0.04, 0.04, 0.95, 0.95])  
    
    figure.savefig(plot_save_path, dpi=400)


